In [1]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [2]:
import sys
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy as sp
from typing import Mapping  
import mmh3
from itertools import chain  
import sharedmem

In [3]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from nearest_neighbors import HNSW


/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [10]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  
import random
import secrets
max_n_neighbors = 20
MAX_SAMPLE_SIZE = int(1e9)
k_values = np.arange(2, max_n_neighbors + 1)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)


In [5]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

read_ids = np.array(list(read_features))

In [6]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(12926, 324674, 25276, 2)

In [11]:
##法一：用secret函数 kmer_index 直接作为种子
def _hash(kmer_index: int) -> np.ndarray:
    random.seed(kmer_index)
    random_number = secrets.token_bytes(400)
    random_integer = int.from_bytes(random_number, byteorder='big')  
    binary_array = format(random_integer, '03200b')
    hash_array = list(binary_array)
    return hash_array
def _get_table(
    kmer_num: list) -> Mapping[int,list]:
    hash_table = np.empty((kmer_num,3200),dtype=np.int8) 
    for kmer_index in range(kmer_num):
        hash_array = _hash(kmer_index)
        hash_table[kmer_index,:]=hash_array
    hash_table = np.where(hash_table == 0, -1, 1) 
    return hash_table
hash_table = _get_table(feature_matrix.shape[1])

In [12]:
simhash = feature_matrix@hash_table
reads_simhash_array = np.where(simhash > 0, 1, 0)

In [13]:
pip install pynear

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 1.9 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [16]:
reads_simhash_array.astype(np.int8)

array([[1, 1, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 0, 1]], dtype=int8)

In [75]:
import pynear
vptree = pynear.VPTreeBinaryIndex()
vptree.set(reads_simhash_array.astype(np.uint8))
vptree_indices, vptree_distances = vptree.searchKNN(reads_simhash_array.astype(np.uint8), 21)
vp_nbr= np.array(vptree_indices)[:,:-1][:,::-1]

In [76]:
vp_nbr

array([[11900,  1764,  5988, ..., 12076,  6839,  1711],
       [11901,  5989,  5504, ...,  8245,  3398,  3230],
       [  598, 12019,  2068, ...,  9687,  4399,  6843],
       ...,
       [ 8636,  9267,  8123, ...,  4184, 12577,  5141],
       [ 1890,  3291, 11969, ...,  9363, 11337,  3040],
       [ 1891,  3290, 11968, ...,  9362, 11336,  3041]])

In [77]:
test= nbr_indices == vp_nbr2

In [78]:
np.sum(test==False)

13138

In [68]:
np.sum(test)

245382

In [79]:
df_rows = []
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=vp_nbr,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'method2', "n_neighbors": k, "repeat_time": 1,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [80]:
df

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,component_sizes
0,method2,2,1,0.991793,0.329497,0.061040,0.260484,123,0.009516,207,"[777, 567, 551, 433, 423, 407, 345, 325, 310, ..."
1,method2,3,1,0.990384,0.289733,0.089459,0.336169,77,0.005957,1255,"[2791, 2093, 1286, 1255, 1206, 1201, 1163, 115..."
2,method2,4,1,0.989292,0.262276,0.117524,0.400222,58,0.004487,3954,"[5162, 3954, 1289, 1286, 1206, 1, 1, 1, 1, 1, ..."
3,method2,5,1,0.988413,0.241740,0.145025,0.455610,50,0.003868,3955,"[6450, 3955, 2496, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,method2,6,1,0.987297,0.225527,0.171874,0.504312,48,0.003713,6451,"[6453, 6451, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,method2,7,1,0.986052,0.211260,0.198575,0.546487,43,0.003327,6452,"[6455, 6452, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,method2,8,1,0.984766,0.199379,0.224782,0.584586,42,0.003249,6453,"[6455, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,method2,9,1,0.983300,0.189789,0.250630,0.621380,39,0.003017,6453,"[6455, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,method2,10,1,0.981885,0.180797,0.276299,0.653505,37,0.002862,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,method2,11,1,0.980635,0.173274,0.301333,0.683929,36,0.002785,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [33]:
np.random.seed(seed=42)

dimension = 32 # 32 bytes are 256 bit dimensional vectos
num_points = 2021
data = np.random.normal(scale=255, loc=0, size=(num_points, dimension)).astype(dtype=np.uint8)

num_queries = 8
queries = np.random.normal(scale=255, loc=0, size=(num_queries, dimension)).astype(dtype=np.uint8)

vptree = pynear.BKTreeBinaryIndex()
vptree.set(data)

# To search using maximum threshold use dimension * 8 (the maximum distance) or set any other threshold
indices, distances, keys = vptree.find_threshold(data, dimension * 8)

In [45]:
indices[1] == indices[2]

True

In [59]:
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

In [29]:
df_rows = []
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'method2', "n_neighbors": k, "repeat_time": 1,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [48]:
nbr_indices

array([[11900,  1764,  5988, ..., 12076,  6839,  1711],
       [11901,  5989,  5504, ...,  8245,  3398,  3230],
       [  598, 12019,  2068, ...,  5981,  4399,  6843],
       ...,
       [ 8636,  9267,  8123, ...,  4184, 12577,  5141],
       [ 1890,  3291, 11969, ...,  9363, 11337,  3040],
       [ 1891,  3290, 11968, ...,  9362, 11336,  3041]])

In [30]:
df

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,component_sizes
0,method2,2,1,0.991741,0.329212,0.061024,0.260207,124,0.009593,207,"[777, 567, 551, 433, 423, 407, 345, 325, 310, ..."
1,method2,3,1,0.990384,0.289709,0.089456,0.336129,78,0.006034,1255,"[2791, 2093, 1285, 1255, 1206, 1201, 1163, 115..."
2,method2,4,1,0.989292,0.262620,0.117521,0.400736,59,0.004564,3954,"[5162, 3954, 1289, 1286, 1206, 1, 1, 1, 1, 1, ..."
3,method2,5,1,0.988453,0.241728,0.145013,0.455531,50,0.003868,3955,"[6450, 3955, 2496, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,method2,6,1,0.987347,0.225588,0.171849,0.504352,48,0.003713,6451,"[6453, 6451, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,method2,7,1,0.986080,0.211258,0.198553,0.546408,43,0.003327,6452,"[6455, 6452, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,method2,8,1,0.984737,0.199347,0.224752,0.584428,42,0.003249,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,method2,9,1,0.983302,0.189664,0.250651,0.621024,38,0.002940,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,method2,10,1,0.981938,0.180780,0.276289,0.653387,37,0.002862,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,method2,11,1,0.980626,0.173247,0.301342,0.683850,36,0.002785,6453,"[6456, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [24]:
len(vptree_indices[1])

20

In [18]:
np.random.seed(seed=42)

dimension = 32 # 32 bytes are 256 bit dimensional vectos
num_points = 2021
data = np.random.normal(scale=255, loc=0, size=(num_points, dimension)).astype(dtype=np.uint8)

num_queries = 8
queries = np.random.normal(scale=255, loc=0, size=(num_queries, dimension)).astype(dtype=np.uint8)

k = 2

vptree = pynear.VPTreeBinaryIndex()
vptree.set(data)
vptree_indices, vptree_distances = vptree.searchKNN(queries, k)

In [20]:
data

array([[126, 221, 165, ..., 182, 103, 216],
       [253, 243, 209, ..., 209, 230, 207],
       [207,  89, 238, ..., 173, 157, 139],
       ...,
       [ 79, 145, 223, ...,   0, 162,  59],
       [ 68,   9,  84, ...,  30, 218,  44],
       [ 34, 107, 189, ...,  17,  79, 247]], dtype=uint8)

In [ ]:
import vptree
tree = vptree.VPTree(reads_simhash_array, hamming_distance)
tree.get_n_nearest_neighbors(list(reads_simhash_array[1]), 10)

In [56]:


#df_tfidf.to_csv('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/human/HLA/ONT_R9/kmer_k16/SimHash_None_None_overlap_stat.tsv',sep='\t')

In [12]:
pip install vptree

  Using cached vptree-1.3.tar.gz (4.4 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for vptree: filename=vptree-1.3-py3-none-any.whl size=4546 sha256=5cc08ad90292e727e0c327e5ad689a94de66da6853727abeac950b9ffbd385d8
  Stored in directory: /home/miaocj/.cache/pip/wheels/3a/eb/ca/904b67530bad0ab02a61978e03549e80bfb1ae07363edb3aab
Successfully built vptree
Note: you may need to restart the kernel to use updated packages.


In [30]:
reads_simhash_array

array([[0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 1]])

In [36]:
points = np.random.randn(20000, 10)
query = [.5] * 10
reads_simhash_array[0]

array([0, 0, 1, ..., 1, 0, 0])

In [47]:
len(reads_simhash_array)

12926

In [ ]:
import numpy as np
import vptree

# Define distance function.
def euclidean(p1, p2):
  return np.sqrt(np.sum(np.power(p2 - p1, 2)))

# Generate some random points.
points = np.random.randn(12926, 3200)
query = [.5] * 3200

# Build tree in O(n log n) time complexity.
tree = vptree.VPTree(points, hamming_distance)

# Query single point.
tree.get_nearest_neighbor(query)

In [57]:
##法一 ：哈希长度为1028
df
df.to_csv('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/human/HLA/ONT_R9/kmer_k16/SimHash_method1_overlap_stat.tsv',sep='\t')

In [50]:
df

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,component_sizes
0,SimHash_TF-IDF,2,100,0.959684,0.290254,0.059900,0.232711,565,0.043710,455,"[857, 725, 718, 686, 665, 615, 557, 514, 463, ..."
1,SimHash_TF-IDF,3,100,0.953179,0.260759,0.086906,0.305389,457,0.035355,1514,"[3303, 2452, 1514, 1178, 1063, 880, 596, 588, ..."
2,SimHash_TF-IDF,4,100,0.947717,0.237915,0.113280,0.365287,394,0.030481,6223,"[6362, 6223, 134, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1..."
3,SimHash_TF-IDF,5,100,0.942152,0.219366,0.138751,0.414979,363,0.028083,6366,"[6366, 6366, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ..."
4,SimHash_TF-IDF,6,100,0.936710,0.205125,0.163604,0.460199,336,0.025994,6369,"[6373, 6369, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, ..."
5,SimHash_TF-IDF,7,100,0.931507,0.193302,0.188038,0.501226,320,0.024756,6376,"[6377, 6376, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,SimHash_TF-IDF,8,100,0.926592,0.182611,0.211843,0.536279,305,0.023596,6379,"[6381, 6379, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,SimHash_TF-IDF,9,100,0.921660,0.173825,0.235100,0.569552,289,0.022358,6382,"[6383, 6382, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,SimHash_TF-IDF,10,100,0.916555,0.165750,0.257791,0.598829,280,0.021662,6385,"[6391, 6385, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,SimHash_TF-IDF,11,100,0.911067,0.158549,0.279779,0.625415,270,0.020888,6385,"[6393, 6385, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
ref_reads_simhash_array = get_simhash(ref_read_features,hash_table)

nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

In [35]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_table(
    read_features: list,  
    feature_matrix: list,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  

    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag,:]=_hash(kmer_index, seed=seed)
            new_hash_table=np.reshape(hash_table,(kmer_num,3200))
    return new_hash_table

def _get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    all_read_simhash = []
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        weighted_hash_table = hash_table*idf[:, np.newaxis]
    else:
        weighted_hash_table = hash_table
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(weighted_hash_table[read_kmer,:],axis=0)
        simhash = np.where(one_read_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)

    return reads_simhash_array 

hash_table = _get_table(read_features,feature_matrix,seed = 15232,tf = False,idf=False,repeat=100)
print("1")
reads_simhash_array = _get_simhash(read_features,feature_matrix,hash_table,seed = 15232,tf = True,idf=False,repeat=100)

In [6]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  
def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty((kmer_num,32), dtype=np.int8) 
            for kmer_index in range(kmer_num):
                result[kmer_index,:]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i,:] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table
hash_table = mp_get_hashtable(read_features,feature_matrix,15232,10)

In [19]:
one_read_hash = np.sum(hash_table[list(read_features.values())[0],:,:],axis=0)
hash_table

In [26]:
hash_table.shape

In [34]:
kmer_num = feature_matrix.shape[1]
hash_table=np.reshape(hash_table,(kmer_num,3200))

In [30]:
len(new_hash[1])

In [18]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

In [19]:
read_ids = np.array(list(read_features))

In [20]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty(kmer_num, dtype=object) 
            for kmer_index in range(kmer_num):
                result[kmer_index]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

## multi-process calculate simhash value
def mp_get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    tf:bool,
    idf:bool,
    processes:int,):
    
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()} 

       
    reads_simhash_array = np.empty((len(read_features),hash_table.shape[1]*32),dtype=object)
    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            read_kmer = list(read_features.values())[i]
            one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
            conc_hash = np.concatenate(one_read_hash)
            result = np.where(conc_hash > 0, 1, 0)
            return i,result

        def reduce(i, result):
            reads_simhash_array[i,:] = result            
        pool.map(work, range(len(read_features)), reduce=reduce)
    return reads_simhash_array

In [21]:
hash_table = mp_get_hashtable(feature_matrix,repeat =100, seed = 4829,processes=12)
print('hashtable established')

In [16]:
df_tfidf

In [111]:
df_tfidf

In [ ]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3

for repeat in [100]:
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829,tf = True,idf = True)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)

In [ ]:
repeat = 100

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

all_read_simhash = []
rng = np.random.default_rng(4829)  
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)
kmer_num = feature_matrix.shape[1]
hash_table = np.empty((kmer_num,repeat),dtype=object)  
for flag,seed in enumerate(hash_seeds):
    print(flag)
    kmer_hash_indice = {} 
    for kmer_index in range(kmer_num):
        hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)
new_hash = hash_table*idf[:, np.newaxis]

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(new_hash[read_kmer,:],axis=0)
    conc_hash = np.concatenate(one_read_hash)
    simhash = np.where(conc_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)

In [ ]:
reads_simhash_array

In [ ]:
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

read_ids = np.array(list(read_features))
k_values = np.arange(2, max_n_neighbors + 1)
df_rows = []

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##带tf idf 初始版本
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    tf:bool,
    idf:bool,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    all_read_simhash = []
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:set(v) for k,v in read_features}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
  
    return reads_simhash_array 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

In [ ]:
new[3][1]

In [ ]:
df

In [ ]:
##32位，70个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  

    hash_seeds = np.array(range(repeat))
    kmer_num = feature_matrix.shape[1]
    kmer_index = np.array(range(kmer_num))

    table = []
    for seed in hash_seeds:
        prime1 = 2654435761  # A large prime number
        prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
        hash_value = (kmer_index * prime1) ^ (seed * prime2)
        hash_t = hash_value % (2**32)
        binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
        one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
        table.append(one_repear_table.astype(int))
    hash_table = np.hstack(table)

    all_read_simhash = []
    for read_ind,read_kmer in read_features.items():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    concat_simhash = np.array(all_read_simhash)
    return concat_simhash 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

dfs = []
for repeat in range(20,110,10):
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graphs[k] = graph

    df_rows = []
    for k in k_values:
        graph = graphs[k]
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)
    
new = pd.concat(dfs)
new.to_csv('/home/miaocj/docker_dir/test_simhash.csv',sep='\t')

In [ ]:
new

In [ ]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [ ]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graphs[k] = graph

In [ ]:
df_rows = []

for k in k_values:
    graph = graphs[k]
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, 
                 **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##70
df

In [ ]:
##40
df

In [ ]:
##20
df

测试hash方法

In [ ]:
import hashlib

def simhash(features):
  
  # Generate a hash for each feature
  hashes = [hashlib.sha1(feature).hexdigest() for feature in features]
  
  # Combine the feature hashes to produce the final simhash
  concatenated_hash = ''.join(hashes)
  simhash = hashlib.sha1(concatenated_hash).hexdigest()
  
  return simhash

In [ ]:
x

In [ ]:
pip install xxhash

In [ ]:
import xxhash
x = np.random.rand(1024 * 1024 * 16)
h = xxhash.xxh64()
h.update(x); h.intdigest(); h.reset()

In [ ]:
x

In [ ]:
## 64位， 40个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:064b}".format(hash_value & 0xFFFFFFFFFFFFFFFF)
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index)  
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 40, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]


In [ ]:
rng = np.random.default_rng(seed)
beta = rng.uniform(0, 1, (feature_count, dimension_count))
x = rng.uniform(0, 1, (feature_count, dimension_count))
u1 = rng.uniform(0, 1, (feature_count, dimension_count))
u2 = rng.uniform(0, 1, (feature_count, dimension_count))

for j_sample in range(0, sample_count):
    feature_indices = sparse.find(data[:, j_sample] > 0)[0]
    gamma = -np.log(np.multiply(u1[feature_indices, :], u2[feature_indices, :]))
    t_matrix = np.floor(
        np.divide(
            matlib.repmat(
                np.log(data[feature_indices, j_sample].todense()),
                1,
                dimension_count,
            ),
            gamma,
        )
        + beta[feature_indices, :]
    )
    y_matrix = np.exp(np.multiply(gamma, t_matrix - beta[feature_indices, :]))
    a_matrix = np.divide(
        -np.log(x[feature_indices, :]),
        np.divide(y_matrix, u1[feature_indices, :]),
    )

In [ ]:
rng = np.random.default_rng(12)

In [ ]:
##建立hash矩阵
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    hash_table = np.empty((repeat,feature_matrix.shape[1]))  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]): 
            hash_table[s,kmer_index] = _hash(kmer_index, seed=s)
    for 
         
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 70, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]